In [1]:
import sys
sys.path.append('..')

### Set OpenAI key 

In [2]:
import os
import configparser

config = configparser.ConfigParser()
config.read('../../.secrets.ini')
OPENAI_API_KEY = config['OPENAI']['OPENAI_API_KEY']
YOUTUBE_KEY = config['YOUTUBE']['YOUTUBE_API_KEY']
NAVER_CLIENT_ID = config['NAVER']['NAVER_CLIENT_ID']
NAVER_CLIENT_SECRET = config['NAVER']['NAVER_CLIENT_SECRET']
GOOGLE_SEARCH_KEY = config['GOOGLE']['GOOGLE_API_KEY']
CSE_ID = config['GOOGLE']['CSE_ID']
SERPAPI_API_KEY = config['SERPAPI']['SERPAPI_API_KEY']


os.environ.update({'OPENAI_API_KEY': OPENAI_API_KEY})
os.environ.update({'YOUTUBE_KEY': YOUTUBE_KEY})
os.environ.update({'NAVER_CLIENT_ID': NAVER_CLIENT_ID})
os.environ.update({'NAVER_CLIENT_SECRET': NAVER_CLIENT_SECRET})
os.environ.update({'GOOGLE_SEARCH_KEY': GOOGLE_SEARCH_KEY})
os.environ.update({'CSE_ID': CSE_ID})
os.environ.update({'SERPAPI_API_KEY': SERPAPI_API_KEY})

### Get tools

In [3]:
# %load_ext autoreload
# %autoreload 2
%reload_ext autoreload

In [4]:
from importlib import reload
import tools
reload(tools)

<module 'tools' from 'c:\\Users\\chois\\Desktop\\OpenAI_SKT\\openai_skt\\tutorials\\..\\tools\\__init__.py'>

In [5]:
from tools import DatabaseTool, SearchTool, SearchByURLTool, TimeTool
from models.llm.chain import SummaryChunkChain

In [6]:
summary_chunk_template = '''
I want you to answer the question from the given documentation. 
There may be a lot of information in a given document that is not relevant to your question, but you should try to find as much relevant information as possible.
Be as diligent as possible in returning your answer based on your knowledge and documentation.
If there is no answer in the document, return "No useful information".

Document: {document}
Question: {question}
Answer:
'''

In [7]:
summary_chunk_chain = SummaryChunkChain(summary_chunk_template=summary_chunk_template, input_variables=["question", "document"], verbose=True)
search_by_url_tool = SearchByURLTool()

In [8]:
from langchain.agents import Tool
from langchain.utilities import SerpAPIWrapper

# You can create the tool to pass to an agent
search = SerpAPIWrapper(params={
        "engine": "google",
        "google_domain": "google.com",
        "gl": "kr",
        "hl": "ko",
        })
search_tool = Tool(
    name="search_tool",
    description="A tool to search on internet.",
    func=search.run,
)

In [9]:
database_tool = DatabaseTool(summary_chunk_chain=summary_chunk_chain)
search_tool = SearchTool(summary_chunk_chain=summary_chunk_chain, search_by_url_tool=search_by_url_tool)
time_tool = TimeTool()

In [10]:
tools = [database_tool, search_tool, time_tool]

In [11]:
from embedchain.embedchain import EmbedChain
from embedchain.config import AppConfig
embed_chain = EmbedChain(config=AppConfig())

In [12]:
from database import DataBase

In [13]:
database = DataBase.load(database_path='./user/test_2/database.json', embed_chain=embed_chain) # optimize: 5m22s # 4m18s 3m4s 3m2s

All data from https://kosis.kr/statHtml/statHtml.do?orgId=201&tblId=DT_201004_O160002_2020&vw_cd=MT_ZTITLE&list_id=201_10_004_016&scrId=&seqNo=&lang_mode=ko&obj_var_id=&itm_id=&conn_path=K1&path=%25EC%25A7%2580%25EC%2597%25AD%25ED%2586%25B5%25EA%25B3%2584%2520%253E%2520%25EC%25A7%2580%25EC%259E%2590%25EC%25B2%25B4%2520%25EA%25B8%25B0%25EB%25B3%25B8%25ED%2586%25B5%25EA%25B3%2584%2520%253E%2520%25EC%2584%259C%25EC%259A%25B8%25ED%258A%25B9%25EB%25B3%2584%25EC%258B%259C%2520%253E%2520%25EC%2584%259C%25EC%259A%25B8%25ED%258A%25B9%25EB%25B3%2584%25EC%258B%259C%25EA%25B8%25B0%25EB%25B3%25B8%25ED%2586%25B5%25EA%25B3%2584%2520%253E%2520%25EA%25B3%25B5%25EA%25B3%25B5%25ED%2596%2589%25EC%25A0%2595%25C2%25B7%25EC%2582%25AC%25EB%25B2%2595%25EC%258B%259C%25EB%25B3%25B8%25EC%25B2%25AD%2520%25EA%25B3%25B5%25EB%25AC%25B4%25EC%259B%2590%2528%25EC%25A0%2595%25EC%259B%2590%2529%25282020%25EB%2585%2584%2520%25EC%259D%25B4%25ED%259B%2584%2529 already exists in the database.
All data from https://kosis.kr/st

In [14]:
import pickle

with open("database_save_path.pkl", "wb") as f:
    pickle.dump(database, f)

with open("database_save_path.pkl", "rb") as f:
    loaded_db = pickle.load(f)


### Set Agent

In [15]:
from models.llm.qna_agent import QnAAgent

In [16]:
qna_agent = QnAAgent(tools=tools, verbose=True, qna_prompt_path='../../openai_skt/models/templates/qna_prompt_template.txt', model='gpt-4')

In [17]:
answer = qna_agent.run(database=database, query="오늘 제주도 날씨가 어때요?")



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. Use a variety of tools to answer the questions. You have access to the following tools:

database: A tool to get data with from database. The input consists of a 'query' and a 'question', where 'query' is the search query and 'question' is the information you want to get. For example, {'query': 'bitcoin price history', 'question': 'what was the price of bitcoin in 2022?'} would be the input if you want to know the price of bitcoin in 2022. Input must contain both a query and a question.
search_tool: A tool to search on internet. The input consists of a 'query' and a 'question', where 'query' is the search query and 'question' is the information you want to get. For example, {'query': 'bitcoin price history', 'question': 'what was the price of bitcoin in 2022?'} would be the input if you want to know the price of bitcoin in 2022. Input must 

In [18]:
from langchain.agents import load_tools
tools = load_tools(["serpapi"])

In [19]:
search_tool.search(query="비트코인 전망", category='google')

{'google_search': [{'title': '비트코인 3500만 원대 공방, 비트코인 하락세 멈추고 4700 ...',
   'description': '[비즈니스포스트]비트코인 시세가 3500만 원대에 머물렀다.비트코인 가격 하락세가 끝나고 2024년 반감기에는 4700만 원대로 올라설 것이라는 전망이 ...',
   'data_type': 'web_page',
   'data_path': 'https://www.businesspost.co.kr/BP?command=article_view&num=325199'},
  {'title': "'9월 하락 징크스' 비트코인, 이더리움과 리플 전망은?",
   'description': '비트코인(Bitcoin, BTC)은 8월 이후 12.3% 하락한 2만 5700달러, ... 연중 고점을 회복한 순간 단기 보유자 투자 심리 회복이 기대된다고 전망했다.',
   'data_type': 'web_page',
   'data_path': 'https://www.blockmedia.co.kr/archives/364701'}]}